<a href="https://colab.research.google.com/github/TAUforPython/2024-SkillFactory-NeuralNetworks/blob/main/scetch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

•	0 - Офтальмологические медицинские изделия\
•	1 - Медицинские изделия для манипуляций/восстановления тканей/органов человека\
•	2 - Сердечно-сосудистые медицинские изделия\
•	3 - Урологические медицинские изделия\
•	4 - Хирургические инструменты/системы и сопутствующие медицинские изделия\
•	5 - Медицинские изделия для пластической хирургии и косметологии\
•	6 - Ортопедические медицинские изделия\
•	7 - Стоматологические медицинские изделия


In [ ]:
!pip install --upgrade gensim
import os
os.kill(os.getpid(), 9)

In [3]:
! pip install pymorphy3 --quiet

In [4]:
!wget 'https://drive.google.com/uc?export=download&id=17wVn5YPpMjHToctGgff_KfSeWcIIlf7c' -O stopwords.txt --quiet

In [5]:
!wget http://vectors.nlpl.eu/repository/20/220.zip --quiet

In [6]:
! unzip /content/220.zip

Archive:  /content/220.zip
replace meta.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: meta.json               
replace model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: model.bin               y

replace model.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: model.txt               y

replace README? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: README                  


https://github.com/Sinaptik-AI/pandas-ai/issues/1251

In [16]:
import pandas as pd
import numpy as np
import gensim
from gensim.models import Word2Vec, KeyedVectors
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from pymorphy3 import MorphAnalyzer
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
# from sklearn.preprocessing import LabelEncoder
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
# Загрузка данных из Excel
file_path = '/content/med_device_supply.xlsx'
model_path = '/content/model.bin'
df = pd.read_excel(file_path)

# Инициализация лемматизатора
morph = MorphAnalyzer()

# Загрузка предобученной модели Word2Vec
word2vec_model = KeyedVectors.load_word2vec_format(model_path, binary=True)


In [25]:
# Предобработка текста.

df = df.drop_duplicates()

stopword_ru = stopwords.words('russian')
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]

stopword_ru += additional_stopwords

# Токенизация и очистка текста.
def prepare_and_tokenize(text):
    text = str(text)
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub("\s+", " ", text)
    text = text.strip()
    text = [word for word in text if word.isalnum() and word not in stopword_ru]
    text = ' '.join(text)
    tokens = word_tokenize(text)

    return  tokens

# Лемматизация. Векторизация.
def vectorize(tokens, analyzer, model):
    keys = []
    for token in tokens:
        parsed = morph.parse(token)
        if len(parsed) == 0:
            continue
        nf = parsed[0].normal_form
        pos = parsed[0].tag.POS
        key = f"{nf}_{pos}"
        keys.append(key)

    vector = pd.Series(model.get_mean_vector(keys))

    return vector

# Кодирование меток
# label_encoder = LabelEncoder()
# df['label'] = label_encoder.fit_transform(df['label'])

In [33]:
df.columns

Index(['med_device_supply', 'Unnamed: 1'], dtype='object')

In [34]:
df['med_device_supply'] = df['med_device_supply'].apply(lambda x: prepare_and_tokenize(x))
df['med_device_supply'] = df['med_device_supply'].apply(lambda x: vectorize(x, morph, word2vec_model)).to_numpy()

In [35]:
df.head()

,med_device_supply,Unnamed: 1
0,-0.029389,Тип
1,-0.029389,Медицинское устройство
2,-0.029389,Ортопедический имплантат
3,-0.029389,Ортопедический имплантат
4,-0.029389,Хирургический материал


In [37]:
# Разделение данных на обучающую и тестовую выборки
X = df.drop('Unnamed: 1', axis=1)

y = df['Unnamed: 1']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение классификатора
classifier = LogisticRegression(multi_class='auto')
classifier.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = classifier.predict(X_test)

# Оценка качества классификации
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.50
Classification Report:
                               precision    recall  f1-score   support

     Ортопедический имплантат       0.50      1.00      0.67         3
 Офтальмологический имплантат       0.00      0.00      0.00         1
Офтальмологический инструмент       0.00      0.00      0.00         1
  Стоматологический имплантат       0.00      0.00      0.00         1

                     accuracy                           0.50         6
                    macro avg       0.12      0.25      0.17         6
                 weighted avg       0.25      0.50      0.33         6

